<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a2a39f1b6bc4973d3b0d86b9e129e4de9dfe238936fb995aa8e8305a87076a9b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 13:11:57
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.43 C
-------------------
Today PnL: -24.97 K (-0.17%)
Current PnL: -22.44 L (-14.7%)
CY Booked + Current PnL: -9.25 L (-6.06%)
-------------------
Total profit:  1.59 L
Total loss:  -24.03 L
-------------------
Total Booked + Current PnL: 17.73 L (14.11%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.24%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 89.90 L (63.01%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.20,86827.0,-13950.0,14040.0,0.33,-13.84,16.17,0.09,245.0,-0.99,0.61,14.07,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.84,55.0,X-MC,6.61,186270.0,11950.0,23246.0,-1.11,6.86,12.48,20.19,91.0,0.51,1.31,23.01,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-3.90,59.0,X-LC,8.21,295776.0,30350.0,26442.0,0.09,11.43,8.94,21.40,11.0,1.15,2.09,18.84,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.0,280.25,69.0,L-MC,7.39,46692.0,-30514.0,55909.0,2.48,-39.52,119.74,32.89,258.0,-0.55,0.33,28.58,XR,NTT,BANKS
13,BERGEPAINT,680.0,-17.54,56.0,X-MC,1.10,221697.0,-5642.0,53695.0,2.20,-2.48,24.22,21.14,106.0,-0.11,1.56,24.96,XY24,NTT,PAINTS
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145706.0,-28853.0,28879.0,1.70,-16.53,19.82,0.02,221.0,-1.00,1.03,26.74,OX40N,NTT,PAINTS
76,TITAGARH,1548.0,-8.23,44.0,H-SC,3.95,196194.0,-45983.0,156759.0,1.51,-18.99,79.90,45.74,158.0,-0.29,1.38,27.60,XY24,NTT,ENGINEERING
35,ICICIPRULI,790.0,-18.13,65.0,X-MC,0.92,182642.0,5904.0,48820.0,1.39,3.34,26.73,30.97,107.0,0.12,1.29,16.32,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,196.51,36.0,M-SC,5.22,132297.0,-50781.0,129757.0,-2.87,-27.74,98.08,43.14,230.0,-0.39,0.93,24.44,XY24,NTT,MISC
83,VOLTAS,1530.00,-6.39,36.0,X-MC,4.03,196290.0,4548.0,33212.0,-2.39,2.37,16.92,19.69,99.0,0.14,1.39,9.33,XY25,NTT,AC
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115524.0,726.0,93320.0,-2.30,0.63,80.78,81.92,223.0,0.01,0.82,36.76,XR,NTT,DURABLES
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248460.0,-5436.0,87731.0,-2.29,-2.14,35.31,32.42,105.0,-0.06,1.75,12.63,X40,ATH,FMCG
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47915.0,-17091.0,168019.0,-2.28,-26.29,350.66,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-44.33,39.0,M-SC,8.43,115524.0,726.0,93320.0,-2.3,0.63,80.78,81.92,223.0,0.01,0.82,36.76,XR,NTT,DURABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.20,86827.0,-13950.0,14040.0,0.33,-13.84,16.17,0.09,245.0,-0.99,0.61,14.07,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145706.0,-28853.0,28879.0,1.70,-16.53,19.82,0.02,221.0,-1.00,1.03,26.74,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,41.0,H-SC,2.28,219924.0,-49743.0,86078.0,-0.59,-18.45,39.14,13.47,138.0,-0.58,1.55,11.83,XY24,NTT,PAINTS
67,SIS,528.0,2026.42,48.0,H-SC,2.19,85820.0,-25212.0,48823.0,0.54,-22.71,56.89,21.26,156.0,-0.52,0.61,15.95,OX40N,NTT,MISC
43,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97922.0,-29883.0,29866.0,0.02,-23.38,30.50,-0.01,232.0,-1.00,0.69,10.88,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,57.18,69.0,M-MC,10.48,247796.0,22834.0,144812.0,-0.30,10.15,58.44,74.53,192.0,0.16,1.75,45.68,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210692.0,5616.0,106947.0,-0.05,2.74,50.76,54.89,52.0,0.05,1.49,27.49,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115524.0,726.0,93320.0,-2.30,0.63,80.78,81.92,223.0,0.01,0.82,36.76,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.89,48.0,H-SC,5.06,128626.0,5290.0,126336.0,-1.02,4.29,98.22,106.72,119.0,0.04,0.91,28.67,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,53.0,M-SC,35.13,158580.0,9360.0,134825.0,-1.70,6.27,85.02,96.62,208.0,0.07,1.12,71.62,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.16,53.0,M-LC,9.17,210692.0,5616.0,106947.0,-0.05,2.74,50.76,54.89,52.0,0.05,1.49,27.49,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.89,48.0,H-SC,5.06,128626.0,5290.0,126336.0,-1.02,4.29,98.22,106.72,119.0,0.04,0.91,28.67,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.33,39.0,M-SC,8.43,115524.0,726.0,93320.0,-2.30,0.63,80.78,81.92,223.0,0.01,0.82,36.76,XR,NTT,DURABLES
85,WIPRO,420.00,-15.15,47.0,M-LC,5.68,149426.0,-1519.0,110979.0,0.49,-1.01,74.27,72.51,53.0,-0.01,1.05,4.96,XR,NTT,IT
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.80,122963.0,-2334.0,61260.0,0.75,-1.86,49.82,47.03,141.0,-0.04,0.87,53.96,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248460.0,-5436.0,87731.0,-2.29,-2.14,35.31,32.42,105.0,-0.06,1.75,12.63,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
45,JSWINFRA,342.00,-24.37,29.0,X-MC,4.43,189979.0,-10368.0,40864.0,0.95,-5.17,21.51,15.23,178.0,-0.25,1.34,25.41,X40N,NTT,REALTY
23,DMART,5391.45,-20.58,30.0,X-LC,6.77,202420.0,-7987.0,67143.0,0.73,-3.80,33.17,28.12,38.0,-0.12,1.43,18.93,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-32.13,34.0,X-MC,0.67,196500.0,-4320.0,72116.0,-1.39,-2.15,36.70,33.76,80.0,-0.06,1.39,2.93,X40,ATH,FMCG
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
66,SIEMENS,4671.50,-6.46,36.0,H-LC,1.64,150480.0,-35615.0,83095.0,-1.47,-19.14,55.22,25.51,15.0,-0.43,1.06,10.89,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.30,36.0,X-MC,3.32,302820.0,-25171.0,131696.0,-1.08,-7.67,43.49,32.48,92.0,-0.19,2.14,2.87,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.24,62.0,X-LC,3.60,245702.0,11558.0,37076.0,1.35,4.94,15.09,20.77,86.0,0.31,1.73,12.20,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47915.0,-17091.0,168019.0,-2.28,-26.29,350.66,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
1,ABB,7934.00,-43.08,31.0,H-LC,5.01,237792.0,-23827.0,143032.0,-0.76,-9.11,60.15,45.57,7.0,-0.17,1.68,1.18,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-7.40,39.0,X-MC,7.14,216320.0,-47045.0,156356.0,-0.37,-17.86,72.28,41.51,84.0,-0.30,1.53,1.51,XY25,ATH,FMCG
52,PAGEIND,51769.93,-29.12,38.0,X-MC,7.64,160220.0,-3520.0,46864.0,-0.36,-2.15,29.25,26.47,82.0,-0.08,1.13,1.51,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,31.0,X-SC,37.27,47915.0,-17091.0,168019.0,-2.28,-26.29,350.66,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.65,41.0,X-SC,4.03,80840.0,-64320.0,140249.0,0.48,-44.31,173.49,52.31,136.0,-0.46,0.57,8.21,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86976.0,-41694.0,86993.0,-0.15,-32.40,100.02,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.14,88905.0,-1773.0,16679.0,0.44,-1.96,18.76,16.44,101.0,-0.11,0.63,17.83,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.48,36.0,X-SC,0.92,106515.0,-21423.0,68553.0,-1.42,-16.75,64.36,36.84,143.0,-0.31,0.75,9.05,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.36,52.0,X-LC,9.36,285290.0,-60666.0,127353.0,0.32,-17.54,44.64,19.28,1.0,-0.48,2.01,5.47,X40,ATH,IT
40,INFY,2275.00,-16.73,62.0,X-LC,4.71,324292.0,11404.0,160298.0,0.59,3.64,49.43,54.87,3.0,0.07,2.29,12.23,X40,BTT,IT
42,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,0.11,-0.80,11.33,10.44,4.0,-0.07,1.40,4.72,X40,NTT,FMCG
82,VBL,671.64,-17.48,50.0,X-LC,5.75,297212.0,-18630.0,131308.0,0.56,-5.90,44.18,35.67,5.0,-0.14,2.10,7.14,X40N,ATH,FMCG
1,ABB,7934.00,-43.08,31.0,H-LC,5.01,237792.0,-23827.0,143032.0,-0.76,-9.11,60.15,45.57,7.0,-0.17,1.68,1.18,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,27.25,77184.0,-36365.0,76412.0,-1.08,-32.03,99.00,35.27,268.0,-0.48,0.54,88.91,XR,NTT,HOTELS
7,ASIANTILES,137.00,7450.00,63.0,L-SC,13.98,82850.0,-10960.0,87581.0,-1.47,-11.68,105.71,81.67,269.0,-0.13,0.58,60.44,XR,NTT,CERAMICS
50,MASFIN,398.61,-20.05,47.0,H-SC,7.91,91065.0,-6915.0,28522.0,-0.48,-7.06,31.32,22.05,152.0,-0.24,0.64,32.37,XR,ATH,FINANCE
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.80,122963.0,-2334.0,61260.0,0.75,-1.86,49.82,47.03,141.0,-0.04,0.87,53.96,MH,ATH,POWER
38,INDIGOPNTS,1408.00,122.10,76.0,M-SC,5.36,145706.0,-28853.0,28879.0,1.70,-16.53,19.82,0.02,221.0,-1.00,1.03,26.74,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.00,67.49,57.0,H-SC,5.62,145258.0,-37517.0,148584.0,-1.77,-20.53,102.29,60.77,125.0,-0.25,1.03,30.82,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7450.00,63.0,L-SC,13.98,82850.0,-10960.0,87581.0,-1.47,-11.68,105.71,81.67,269.0,-0.13,0.58,60.44,XR,NTT,CERAMICS
38,INDIGOPNTS,1408.00,122.10,76.0,M-SC,5.36,145706.0,-28853.0,28879.0,1.70,-16.53,19.82,0.02,221.0,-1.00,1.03,26.74,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.70,73.0,X-LC,6.16,232866.0,-18902.0,72119.0,-0.16,-7.51,30.97,21.13,27.0,-0.26,1.64,25.26,X40,ATH,PAINTS
5,ANGELONE,3033.00,22.45,68.0,X-SC,6.67,216488.0,25482.0,32213.0,1.29,13.34,14.88,30.21,157.0,0.79,1.53,37.01,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.87
1,25,44.83
2,50,76.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.49
LC    30.49
MC    26.00
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.78
X40      19.31
X40N     12.47
XY25     10.23
XR        9.69
AR        7.72
OX40N     6.61
X200      1.80
X5K       1.49
SR        1.01
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    21.60
X-LC    21.01
M-SC    12.50
X-SC     5.58
H-LC     4.54
M-LC     3.94
H-MC     2.32
M-MC     1.75
L-SC     1.45
L-LC     1.00
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.99,-4.54,38.35
IT,12.62,-17.50,80.39
FINANCE,12.17,-9.75,59.51
MISC,7.70,-19.16,79.26
PAINTS,5.78,-12.58,29.36
ELECTRICAL,5.63,-13.29,53.46
INSURANCE,4.48,0.45,34.62
AUTO,2.71,-49.18,111.44
BANKS,2.70,-39.74,130.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3308447.0,21
XR,1282460.0,13
AR,1193314.0,9
X40,953987.0,13
X40N,700618.0,9
OX40N,549107.0,9
XY25,487255.0,7
SR,277099.0,2
X5K,106947.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3438208.0,24
X-MC,1347321.0,15
M-SC,1272880.0,15
X-LC,1061733.0,12
X-SC,557959.0,6
H-LC,295828.0,3
M-LC,265069.0,3
H-MC,257941.0,2
L-SC,254208.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1189587.0      6
           AR         842938.0      5
           XR         809056.0      7
M-SC       XY24       744915.0      6
X-MC       XY24       568457.0      4
X-LC       X40        439842.0      5
X-MC       X40        427152.0      7
X-LC       XY24       308972.0      2
X-SC       X40N       302947.0      4
H-SC       SR         277099.0      2
           OX40N      258268.0      3
X-LC       X40N       235527.0      3
H-LC       AR         226127.0      2
X-MC       XY25       189568.0      2
H-MC       XY24       183685.0      1
X-SC       XY24       168019.0      1
L-SC       XR         163993.0      2
X-MC       X40N       162144.0      2
M-MC       XY24       144812.0      1
M-SC       XR         142523.0      2
           XY25       134825.0      1
           OX40N      126368.0      4
           AR         124249.0      2
M-LC       XR         110979.0      1
           X5K        106947.0      1
L-SC       OX40N       90215.0      1
X-SC       X40         86993.0      1
X-LC       XY25        77392.0      2
H-MC       OX40N       74256.0      1
H-LC       X200        69701.0      1
H-SC       MH          61260.0      1
L-MC       XR          55909.0      1
M-LC       XY25        47143.0      1
L-LC       XY25        38327.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
